In [34]:
import requests
import json
import time
import csv
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
import csv
from selenium import webdriver
import threading
from threading import Thread
import requests_cache
import selenium
import matplotlib.pyplot as plt
import seaborn as sns

sleep_time = 3

# Collect data from API

In [27]:
def collect_repositories(genre,client_id,limit_count,offset_count):
    repositories = []
    time.sleep(sleep_time)
    url_soundcloud=f'https://api-v2.soundcloud.com/search/playlists?q={genre}&client_id={client_id}&limit={limit_count}&offset={offset_count}'
    r= requests.get(url_soundcloud)
    while not r.ok:
        time.sleep(sleep_time)
        r= requests.get(url_soundcloud)   
    data = json.loads(r.text)
    for i in data['collection']:
        if i not in repositories:
            repositories.append(i)
    return repositories 

In [28]:
def collect_all_repositories(genre,client_id,limit_count,min_repositories):
    all_repositories = []
    starting_offset=0
    while len(all_repositories)<min_repositories:
        r=collect_repositories(genre,client_id,limit_count,starting_offset)
        all_repositories.extend(r)
        starting_offset+=limit_count
    return all_repositories

In [29]:
def collect_playlists_to_csv(repositories):
    playlist_id=[]
    playlist_title=[]
    playlist_user_id=[]
    playlist_description=[]
    playlist_url=[]
    playlist_genre=[]
    playlist_username=[]
    playlist_track=[]
    playlist_track_str=''
    playlist_track_count=[]
    playlist_likes_count=[]
    playlist_reposts_count=[]
    for repo in repositories:
        playlist_id.append(repo['id'])
        playlist_title.append(repo['title'])
        playlist_user_id.append(repo['user_id'])
        playlist_description.append(repo['description'])
        playlist_url.append(repo['permalink_url'])
        playlist_genre.append(repo['genre'])
        playlist_username.append(repo['user']['username'])
        head_of_tracks=20 if len(repo['tracks'])>20 else len(repo['tracks'])-1
        for i in range(head_of_tracks):
            playlist_track_str+=str(repo['tracks'][i]['id'])+' '
        playlist_track_count.append(repo['track_count'])
        playlist_likes_count.append(repo['likes_count'])
        playlist_reposts_count.append(repo['reposts_count'])
        playlist_track.append(playlist_track_str)
        playlist_track_str=''
    playlist_frame={
        'id': playlist_id,
        'title': playlist_title,
        'genre': playlist_genre,
        'description': playlist_description,
        'url': playlist_url,
        'user_id': playlist_user_id,
        'username': playlist_username,
        'trackIds':playlist_track,
        'track_count': playlist_track_count,
        'likes_count':playlist_likes_count,
        'reposts_count': playlist_reposts_count
    }
    pl=pd.DataFrame(playlist_frame)
    pl.to_csv('data/playlist.csv', index=False)

In [30]:
def collect_users_to_csv(repositories):
    user_id=[]
    user_city=[]
    user_country=[]
    user_description=[]
    user_followers_count=[]
    user_followings_count=[]
    user_likes_count=[]
    user_playlist_likes_count=[]
    user_url=[]
    user_playlist_count=[]
    user_track_count=[]
    user_username=[]
    for repo in repositories:
        user=repo['user']
        user_id.append(user['id'])
        user_username.append(user['username'])
        user_city.append(user['city'])
        user_country.append(user['country_code'])
        user_description.append(user['description'])
        user_followers_count.append(user['followers_count'])
        user_followings_count.append(user['followings_count'])
        user_likes_count.append(user['likes_count'])
        user_playlist_likes_count.append(user['playlist_likes_count'])
        user_url.append(user['permalink_url'])
        user_playlist_count.append(user['playlist_count'])
        user_track_count.append(user['track_count'])
    user_frame={
        'id': user_id,
        'username': user_username,
        'city': user_city,
        'country' :user_country,
        'description': user_description,
        'followers_count': user_followers_count,
        'followings_count': user_followings_count,
        'likes_count': user_likes_count,
        'playlist_likes_count': user_playlist_likes_count,
        'url': user_url,
        'playlist_count':user_playlist_count,
        'track_count':user_track_count
    }
    us=pd.DataFrame(user_frame)
    us.to_csv('data/user.csv', index=False)

In [31]:
def collect_tracks_to_csv(repositories):
    track_playlist=[]
    track_id=[]
    track_title=[]
    track_user_id=[]
    track_artist=[]
    track_description=[]
    track_comment_count=[]
    track_downloadable=[]
    track_download_count=[]
    track_genre=[]
    track_likes_count=[]
    track_url=[]
    track_playback_count=[]
    track_reposts_count=[]
    for repo in repositories:
        head_of_tracks=10 if len(repo['tracks'])>10 else len(repo['tracks'])-1
        for i in range(head_of_tracks):
            if len(repo['tracks'][i])<5:
                continue
            track_playlist.append(repo['id'])
            track=repo['tracks'][i]
            track_id.append(track['id'])
            track_title.append(track['title'])
            track_user_id.append(track['user_id'])
            if track['publisher_metadata']:
                if 'artist' in track['publisher_metadata']:
                    track_artist.append(track['publisher_metadata']['artist'])
                else: track_artist.append('')
            else: track_artist.append('')
            track_description.append(track['description'])
            track_comment_count.append(track['comment_count'])
            track_downloadable.append(track['downloadable'])
            track_download_count.append(track['download_count'])
            track_genre.append(track['genre'])
            track_likes_count.append(track['likes_count'])
            track_url.append(track['permalink_url'])
            track_playback_count.append(track['playback_count'])
            track_reposts_count.append(track['reposts_count'])
    track_frame={
        'playlist': track_playlist,
        'id': track_id,
        'title': track_title,
        'user_id': track_user_id,
        'artist': track_artist,
        'description': track_description,
        'comment_count': track_comment_count,
        'downloadable': track_downloadable,
        'download_count': track_download_count,
        'genre': track_genre, 
        'likes_count': track_likes_count, 
        'url': track_url, 
        'playback_count': track_playback_count, 
        'reposts_count': track_reposts_count
    }
    tr=pd.DataFrame(track_frame)
    tr.to_csv('Api_data/track.csv', index=False)

In [32]:
def get_data(limit_count, min_repositories, genre):
    client_id='9W64ngrJNaQIXfNm6SORus4z4ucknVhz'
    all_repositories=collect_all_repositories(genre,client_id,limit_count,min_repositories)
    collect_playlists_to_csv(all_repositories)
    collect_users_to_csv(all_repositories)
    collect_tracks_to_csv(all_repositories)

In [ ]:
# get_data(100, 1000, 'lo-fi')